# How to Create Tool-Enabled Agents Using LangGraph and LangChain

## **Introduction**

In the rapidly evolving world of AI and natural language processing, building intelligent agents that can interact with tools and external systems is becoming increasingly important. LangGraph, a powerful framework for creating graph-based workflows, provides a seamless way to integrate tools and language models into agent-based systems. This article explores how key components—`langgraph.prebuilt.ToolNode`, `langchain_community.agent_toolkits.load_tools`, and `langchain.tools.StructuredTool`—work together with `langgraph.prebuilt.create_react_agent` to create dynamic, tool-enabled agents. By understanding these building blocks, developers can design agents that leverage external tools, manage complex workflows, and deliver structured outputs, unlocking new possibilities for intelligent automation.

### **Comparison Table**

| Feature/Class               | `ToolNode`                          | `load_tools`                        | `StructuredTool.from_function`       |
|-----------------------------|-------------------------------------|-------------------------------------|--------------------------------------|
| **Purpose**                 | Executes tools in a graph node      | Loads predefined tools by name      | Converts a function into a tool      |
| **Tool Integration**        | Yes                                 | Yes                                 | Yes                                  |
| **Custom Tools**            | Supported                           | No                                  | Yes                                  |
| **Error Handling**          | Yes                                 | No                                  | No                                   |
| **LLM Integration**         | Optional                            | Optional                            | No                                   |
| **Use Case**                | Executing tools in a workflow       | Loading prebuilt tools              | Creating custom tools                |

---

## Preparation

### Installing Required Libraries

This section installs the necessary Python libraries for working with LangChain, OpenAI, Anthropic, DuckDuckGo, Arxiv, GraphQL, and other utilities. These libraries include:

- **`langchain-openai`**: Provides integration with OpenAI's language models and APIs, enabling the use of models like GPT-4 for natural language tasks.
- **`langchain-anthropic`**: Enables integration with Anthropic's models (e.g., Claude) for advanced language processing and reasoning.
- **`langchain_community`**: Contains community-contributed modules and tools for LangChain, including additional utilities and integrations.
- **`langchain_experimental`**: Includes experimental features and utilities for LangChain, offering cutting-edge capabilities for advanced use cases.
- **`langgraph`**: A library for building and visualizing graph-based workflows in LangChain, essential for creating tool-enabled agents and complex workflows.
- **`duckduckgo-search`**: Enables programmatic access to DuckDuckGo's search functionality, allowing agents to retrieve real-time information from the web.
- **`arxiv`**: Provides access to the Arxiv API, enabling agents to search and retrieve scientific papers and research articles.
- **`httpx` and `gql`**: Libraries for making HTTP requests and interacting with GraphQL APIs, essential for integrating GraphQL-based tools and services.

These libraries form the foundation for building intelligent agents that can interact with external tools, retrieve information, and execute complex workflows.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain_community
!pip install -qU langchain_experimental
!pip install -qU langgraph
!pip install -qU duckduckgo-search
!pip install -qU arxiv
!pip install -qU httpx gql # graphql

### Initializing OpenAI and Anthropic Chat Models
This section demonstrates how to securely fetch API keys for OpenAI and Anthropic using Kaggle's `UserSecretsClient` and initialize their respective chat models. The `ChatOpenAI` and `ChatAnthropic` classes are used to create instances of these models, which can be used for natural language processing tasks such as text generation and conversational AI.

**Key steps:**
1. **Fetch API Keys**: The OpenAI and Anthropic API keys are securely retrieved using Kaggle's `UserSecretsClient`.
2. **Initialize Chat Models**:
   - The `ChatOpenAI` class is initialized with the `gpt-4o-mini` model and the fetched OpenAI API key.
   - The `ChatAnthropic` class is initialized with the `claude-3-5-sonnet-latest` model and the fetched Anthropic API key.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from kaggle_secrets import UserSecretsClient

# Fetch API key securely
user_secrets = UserSecretsClient()

# Initialize LLM
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=user_secrets.get_secret("my-openai-api-key"))
#model = ChatAnthropic(model="claude-3-5-sonnet-latest", temperature=0, api_key=user_secrets.get_secret("my-anthropic-api-key"))

In [ ]:
import json

def pretty_print(step):
    """
    Pretty prints the `step` output as formatted JSON.
    
    Args:
        step: The step output from the agent's stream.
    """
    # Convert the step to a JSON string with indentation
    step_json = json.dumps(step, indent=4, default=str)
    print(step_json)

---

## **Tool Integration**

### **`ToolNode` class**
This class represents a node in a graph that executes tools based on the tool calls in the last `AIMessage`. It runs tools in parallel if multiple tool calls are requested.

**Key Features**:
- Executes tools dynamically based on LLM tool calls.
- Handles tool errors gracefully.
- Can be integrated into a `StateGraph` for complex workflows.

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def check_weather(location: str) -> str:
    """Call to check the current weather."""
    return f"It's always sunny in {location}"

tool_node = ToolNode(tools=[check_weather])

message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "check_weather",
            "args": {"location": "sf"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

result = tool_node.invoke({"messages": [message_with_single_tool_call]})
print(result)

### **`load_tools` function**
This function loads predefined tools by their names. These tools allow agents to interact with external resources like APIs, databases, and file systems.

**Key Features**:
- Loads tools dynamically based on their names.
- Supports optional LLM integration for certain tools.
- Allows enabling dangerous tools (e.g., tools with elevated permissions) with caution.

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools

# Load tools
tools = load_tools(["ddg-search", "graphql", "arxiv"], 
                   llm=model, 
                   graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index")
print(tools)

### **`StructuredTool` class**
This method creates a custom tool from a Python function. It is useful for defining tools with specific input and output schemas.

**Key Features**:
- Converts a function into a tool with a name, description, and schema.
- Supports both synchronous and asynchronous functions.
- Can infer schemas from function signatures and docstrings.

In [ ]:
from langchain_core.tools.structured import StructuredTool

def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

tool = StructuredTool.from_function(add)

# Correct way: Pass arguments as a dictionary
result = tool.run({"a": 1, "b": 2})
print(result)

---

## **Using Built-In Tools**

### **Example 1: Basic Agent with DuckDuckGo and Arxiv**

This example demonstrates how to create a basic agent that uses built-in tools like **DuckDuckGo Search** and **Arxiv** to search for information and summarize results. The agent is initialized with a language model (LLM) and a `ToolNode` that wraps the tools for execution.

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import ToolNode, create_react_agent

# Step 1: Load tools
tools = load_tools(["ddg-search", "arxiv"], llm=model)

# Step 2: Create a ToolNode
tool_node = ToolNode(tools=tools, name="tools")

# Step 3: Create a React Agent
agent = create_react_agent(model, tool_node)

# Step 4: Run the agent
inputs = {"messages": [("user", "Search for recent papers on quantum computing and summarize the top result.")]}
for step in agent.stream(inputs):
    pretty_print(step)

### **Example 2: Interrupts and Checkpointing**

This example demonstrates how to add **interrupts** and **checkpointing** to the agent workflow. Interrupts allow the agent to pause execution before or after specific nodes (e.g., before executing tools), while checkpointing enables the agent to persist its state across interactions.

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Step 1: Load tools and create ToolNode
tools = load_tools(["ddg-search", "arxiv", "graphql"], 
                   llm=model, 
                   graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index")
tool_node = ToolNode(tools=tools, name="tools")

# Step 2: Create a React Agent with interrupts and checkpointing
agent = create_react_agent(
    model,
    tool_node,
    interrupt_before=["tools"],  # Pause before executing tools
    checkpointer=MemorySaver()   # Enable checkpointing for chat memory
)

# Step 3: Run the agent
inputs = {"messages": [("user", "Search for recent papers on quantum computing and summarize the top result.")]}
config = {"configurable": {"thread_id": "thread-1"}}
for step in agent.stream(inputs, config):
    pretty_print(step)

### **Example 3: Cross-Thread Memory**

This example introduces **cross-thread memory** using an `InMemoryStore`. The store allows the agent to persist data across multiple threads or conversations, enabling features like user-specific memory or shared context.

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore

# Step 1: Define a memory store
store = InMemoryStore()

# Step 2: Load tools and create ToolNode
tools = load_tools(["ddg-search", "arxiv", "graphql"], 
                   llm=model, 
                   graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index")
tool_node = ToolNode(tools=tools, name="tools")

# Step 3: Create a React Agent with cross-thread memory
agent = create_react_agent(
    model,
    tool_node,
    store=store,
    checkpointer=MemorySaver()
)

# Step 4: Run the agent
inputs = {"messages": [("user", "Search for recent papers on quantum computing and summarize the top result.")]}
config = {"configurable": {"thread_id": "thread-1", "user_id": "123"}}
for step in agent.stream(inputs, config):
    pretty_print(step)

### **Example 4: Complex Prompt and State Modifier**

This example demonstrates how to use a **state modifier** to customize the input to the LLM. A `ChatPromptTemplate` is used to define a complex prompt, and a state modifier function dynamically prepares the input for the LLM.

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.prompts import ChatPromptTemplate

# Step 1: Define a state modifier
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant named Fred."),
    ("placeholder", "{messages}"),
    ("user", "Remember to be polite!"),
])

def state_modifier(state):
    return prompt.invoke({"messages": state["messages"]})

# Step 2: Load tools and create ToolNode
tools = load_tools(["ddg-search", "arxiv", "graphql"], 
                   llm=model, 
                   graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index")
tool_node = ToolNode(tools=tools, name="tools")

# Step 3: Create a React Agent with state modifier
agent = create_react_agent(
    model,
    tool_node,
    state_modifier=state_modifier
)

# Step 4: Run the agent
inputs = {"messages": [("user", "Search for recent papers on quantum computing and summarize the top result.")]}
for step in agent.stream(inputs):
    pretty_print(step)

---

## **Using Custom Tools**

### **Example 1: Basic Agent with Custom Tools**

This example demonstrates how to create a basic agent using custom tools. We define three custom tools (`get_current_time`, `calculate_tip`, and `get_weather`) using `StructuredTool.from_function`. These tools are then wrapped in a `ToolNode` and integrated into a `create_react_agent` workflow. The agent is initialized with an OpenAI GPT-4 model and processes user queries by dynamically calling the appropriate tool.

In [ ]:
from langchain_core.tools.structured import StructuredTool
from datetime import datetime
import pytz

# Tool 1: Get current time in a specified timezone
def get_current_time(timezone: str) -> str:
    """Returns the current time in the specified timezone."""
    tz = pytz.timezone(timezone)
    current_time = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    return f"The current time in {timezone} is {current_time}."

# Tool 2: Calculate tip amount
def calculate_tip(bill_total: float, tip_percentage: float) -> str:
    """Calculates the tip amount for a given bill total and tip percentage."""
    tip_amount = bill_total * (tip_percentage / 100)
    total_amount = bill_total + tip_amount
    return f"Tip: ${tip_amount:.2f}, Total: ${total_amount:.2f}"

# Tool 3: Get weather for a location (simulated)
def get_weather(location: str) -> str:
    """Simulates fetching the weather for a given location."""
    return f"The weather in {location} is sunny with a high of 75°F."

# Create StructuredTool instances
time_tool = StructuredTool.from_function(get_current_time)
tip_tool = StructuredTool.from_function(calculate_tip)
weather_tool = StructuredTool.from_function(get_weather)

# Combine tools into a list
tools = [time_tool, tip_tool, weather_tool]

# Create a ToolNode
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools=tools, name="tools")

# Create a React Agent
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# Create the agent
agent = create_react_agent(model, tool_node)

# Run the agent
inputs = {"messages": [("user", "What's the current time in New York?")]}
for step in agent.stream(inputs):
    pretty_print(step)

### **Example 2: Interrupts and Checkpointing**

This example demonstrates how to add **interrupts** and **checkpointing** to the agent workflow. Interrupts allow the agent to pause execution before or after specific nodes (e.g., before executing tools), enabling user confirmation or additional processing. Checkpointing, using `MemorySaver`, persists the state of the agent across interactions, enabling features like chat memory. This example is ideal for building interactive and stateful conversational agents.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Create the agent with interrupts and checkpointing
agent = create_react_agent(
    model,
    tool_node,
    interrupt_before=["tools"],  # Pause before executing tools
    checkpointer=MemorySaver()   # Enable checkpointing for chat memory
)

# Run the agent
inputs = {"messages": [("user", "What's the weather in San Francisco?")]}
config = {"configurable": {"thread_id": "thread-1"}}
for step in agent.stream(inputs, config):
    pretty_print(step)

### **Example 3: Cross-Thread Memory**

This example introduces **cross-thread memory** using an `InMemoryStore`. The store allows the agent to persist data across multiple threads or conversations, enabling features like user-specific memory or shared context. The agent retrieves and updates user memories dynamically, enhancing the conversational experience. This example is ideal for multi-user or multi-session applications.

In [ ]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
store = InMemoryStore()

# Create a React Agent with Cross-Thread Memory
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

# Create the agent with cross-thread memory
agent = create_react_agent(
    model,
    tool_node,
    store=store,
    checkpointer=MemorySaver()
)

# Run the agent
inputs = {"messages": [("user", "What's the current time in London?")]}
config = {"configurable": {"thread_id": "thread-1", "user_id": "123"}}
for step in agent.stream(inputs, config):
    pretty_print(step)

### **Example 4: Complex Prompt and State Modifier**

This example demonstrates how to use a **state modifier** to customize the input to the LLM. A `ChatPromptTemplate` is used to define a complex prompt, and a state modifier function dynamically prepares the input for the LLM. This approach is useful for adding system prompts, contextual instructions, or other modifications to the agent's behavior. The example shows how to integrate a state modifier into the agent workflow.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Define a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant named Fred."),
    ("placeholder", "{messages}"),
    ("user", "Remember to be polite!"),
])

# Define a state modifier
def state_modifier(state):
    return prompt.invoke({"messages": state["messages"]})

# Create a React Agent with State Modifier
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# Create the agent with a state modifier
agent = create_react_agent(
    model,
    tool_node,
    state_modifier=state_modifier
)

# Run the agent
inputs = {"messages": [("user", "What's the weather in Tokyo?")]}
for step in agent.stream(inputs):
    pretty_print(step)

---

## **Conclusion**

The integration of `ToolNode`, `load_tools`, and `StructuredTool` with `create_react_agent` provides a robust foundation for building intelligent agents that can interact with tools and external systems. These components enable developers to create agents that are not only capable of understanding and generating natural language but also of executing tasks, retrieving information, and delivering structured responses. Whether you're building a conversational assistant, a research tool, or an automation workflow, understanding how these objects work together empowers you to design sophisticated, tool-enabled agents that can handle real-world challenges. As AI continues to advance, mastering these tools will be key to unlocking the full potential of intelligent systems.